In [189]:
#profitableHospital-project - SVC Model

In [1]:
#dependencies
import pandas as pd

import requests
import json


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

import joblib

In [2]:
#initial file read
df = pd.read_csv("2017targetedmain.csv")
df.head()

,Profit Score,Profit %,CON,Urban,Provider Type,Type of Control,FTE - Employees on Payroll,Number of Beds,Discharges,Cost of Charity Care,...,Outpatient Revenue,Gross Revenue,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev Per Discharge,Rev Per Bed,Discharges Per Bed,Rev Per FTE
0,1,0.122992,1.0,1.0,1,2,322.10,70.0,435.0,26358.0,...,14763631.0,21455064.0,0.311881,0.688119,0.001229,0.001229,49321.986207,3.065009e+05,6.214286,66609.947221
1,3,-0.037184,1.0,2.0,1,9,923.39,155.0,2110.0,782377.0,...,101005874.0,165935062.0,0.391293,0.608707,0.004715,0.004715,78642.209479,1.070549e+06,13.612903,179702.034893
2,3,-0.090105,1.0,2.0,1,2,378.26,49.0,632.0,14670.0,...,44004253.0,60856429.0,0.276917,0.723083,0.000241,0.000241,96291.818038,1.241968e+06,12.897959,160885.182150
3,3,-0.021699,1.0,1.0,1,2,693.11,48.0,1422.0,1989303.0,...,74054548.0,113478041.0,0.347411,0.652589,0.017530,0.017530,79801.716596,2.364126e+06,29.625000,163722.989136
4,2,0.018523,1.0,1.0,1,2,1452.05,268.0,9188.0,9740925.0,...,625072412.0,983465760.0,0.364419,0.635581,0.009905,0.009905,107038.067044,3.669648e+06,34.283582,677294.693709


In [3]:
#df.dtypes

In [4]:
#df['Profit %'] = pd.to_numeric(df['Profit %'])

In [5]:
#df['Profit %'].dtype

In [6]:
#df1 = df.loc[df["Profit %"] <= .0915]

In [7]:
#df2 = df1.loc[df["Profit %"] >= -.0656]

In [8]:
df2 = df

In [9]:
# Select features to be used as x values
features2 = df2[['CON', 'Urban','Provider Type','Type of Control','FTE - Employees on Payroll', 'Number of Beds','Discharges','Cost of Charity Care','Cost of Uncompensated Care','Total Costs', 'Total fixed Assets', 'Total Long Term Liabilities', 'Inpatient Revenue', 'Outpatient Revenue', 'Gross Revenue', 'Inpatient %', 'Outpatient %', 'Charity %', 'Uncompensated %', 'Rev Per Discharge', 'Rev Per Bed', 'Discharges Per Bed']]
features2.head()

,CON,Urban,Provider Type,Type of Control,FTE - Employees on Payroll,Number of Beds,Discharges,Cost of Charity Care,Cost of Uncompensated Care,Total Costs,...,Inpatient Revenue,Outpatient Revenue,Gross Revenue,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev Per Discharge,Rev Per Bed,Discharges Per Bed
0,1.0,1.0,1,2,322.10,70.0,435.0,26358.0,284986.0,7704339.0,...,6691433.0,14763631.0,21455064.0,0.311881,0.688119,0.001229,0.001229,49321.986207,3.065009e+05,6.214286
1,1.0,2.0,1,9,923.39,155.0,2110.0,782377.0,3793100.0,39198497.0,...,64929188.0,101005874.0,165935062.0,0.391293,0.608707,0.004715,0.004715,78642.209479,1.070549e+06,13.612903
2,1.0,2.0,1,2,378.26,49.0,632.0,14670.0,269451.0,20033059.0,...,16852176.0,44004253.0,60856429.0,0.276917,0.723083,0.000241,0.000241,96291.818038,1.241968e+06,12.897959
3,1.0,1.0,1,2,693.11,48.0,1422.0,1989303.0,1882929.0,43205601.0,...,39423493.0,74054548.0,113478041.0,0.347411,0.652589,0.017530,0.017530,79801.716596,2.364126e+06,29.625000
4,1.0,1.0,1,2,1452.05,268.0,9188.0,9740925.0,15188401.0,222474406.0,...,358393348.0,625072412.0,983465760.0,0.364419,0.635581,0.009905,0.009905,107038.067044,3.669648e+06,34.283582


In [10]:
df2['Profit Score'].value_counts()

2    1908
3     954
1     954
Name: Profit Score, dtype: int64

In [11]:
y = df2[['Profit Score']]

In [12]:
X = features2

In [13]:
#Create a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [14]:
y_train.shape

(2862, 1)

In [15]:
X_train.shape

(2862, 22)

In [16]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create the SVC linear regression model
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [18]:
# Tune the model with GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
gridb = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
gridb.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/5] END ..............................C=1, gamma=0.0001; total time=   0.3s


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/5] END ..............................C=1, gamma=0.0001; total time=   0.4s


...

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 3/5] END ..............................C=50, gamma=0.001; total time=   3.3s


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 4/5] END ..............................C=50, gamma=0.001; total time=   4.0s


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
print(gridb.best_params_)
print(gridb.best_score_)

In [ ]:
predictions = gridb.predict(X_test)

In [ ]:
# Calculate classification report
print(classification_report(y_test, predictions,
                            target_names=["3", "2" , "1"]))